In [2]:
# Linux 등의 경우
!export BENTOML_HOME='C:\Users\dqkim\Documents\in2wise\bentoml'

'export'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [4]:
# Window의 경우
!setx BENTOML_HOME 'C:\Users\dqkim\Documents\in2wise\bentoml'


����: ������ ���� �����߽��ϴ�.


# 1. Model Training

In [5]:
from sklearn import svm, datasets

# Load training data
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Model Training
clf = svm.SVC()
clf.fit(X, y)

SVC()

In [6]:
import bentoml
bentoml.sklearn.save_model("iris_clf", clf)

Model(tag="iris_clf:uxekb6pttkm46yhd", path="C:\Users\dqkim\bentoml\models\iris_clf\uxekb6pttkm46yhd\")

In [7]:
# Model List 확인
!bentoml models list

 Tag                        Module           Size      Creation Time       
 iris_clf:uxekb6pttkm46yhd  bentoml.sklearn  5.62 KiB  2022-06-24 17:50:07 
 iris_clf:rawzrvpttg3fmyhd  bentoml.sklearn  5.62 KiB  2022-06-24 17:42:08 
 iris_clf:ntdxd6pttgsvwyhd  bentoml.sklearn  5.62 KiB  2022-06-24 17:41:22 
 iris_clf:nbn45dhttgzpiyhd  bentoml.sklearn  5.62 KiB  2022-06-24 17:41:14 
 iris_clf:zsb53h7ttcgxayhd  bentoml.sklearn  5.62 KiB  2022-06-24 17:36:53 
 iris_clf:s2j5mrhttcljkyhd  bentoml.sklearn  5.62 KiB  2022-06-24 17:35:22 


In [10]:
# 특정 버전 모델 가져오기
!bentoml models get iris_clf:rawzrvpttg3fmyhd

name: iris_clf                                                                 
version: rawzrvpttg3fmyhd                                                      
module: bentoml.sklearn                                                        
labels: {}                                                                     
options: {}                                                                    
metadata: {}                                                                   
context:                                                                       
  framework_name: sklearn                                                      
  framework_versions:                                                          
    scikit-learn: 1.0.2                                                        
  bentoml_version: 1.0.0rc2                                                    
  python_version: 3.9.10                                                       
signatures:                             

In [8]:
# 가장 최신 버전 가져오기
!bentoml models get iris_clf:latest

name: iris_clf                                                                 
version: uxekb6pttkm46yhd                                                      
module: bentoml.sklearn                                                        
labels: {}                                                                     
options: {}                                                                    
metadata: {}                                                                   
context:                                                                       
  framework_name: sklearn                                                      
  framework_versions:                                                          
    scikit-learn: 1.0.2                                                        
  bentoml_version: 1.0.0rc2                                                    
  python_version: 3.9.10                                                       
signatures:                             

# 2. Model Serving - Runner

In [16]:
import bentoml

model1 = bentoml.sklearn.get('iris_clf:latest').to_runner()
model2 = bentoml.sklearn.get('iris_clf:rawzrvpttg3fmyhd').to_runner()

model1.init_local()
model2.init_local()

'Runner.init_local' is for debugging and testing only
'Runner.init_local' is for debugging and testing only


In [17]:
print(f"Model1 Predict : {model1.predict.run([[5.9, 3, 5.1, 1.8]])}")
print(f"Model2 Predict : {model2.predict.run([[5.9, 3, 5.1, 1.8]])}")

Model1 Predict : [2]
Model2 Predict : [2]


# 3. Model Serving - Service

In [18]:
%%writefile service.py

import numpy as np
import bentoml
from bentoml.io import NumpyNdarray

iris_clf_runner = bentoml.sklearn.get("iris_clf:latest").to_runner()

svc = bentoml.Service("iris_classifier", runners=[iris_clf_runner])

@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_series: np.ndarray) -> np.ndarray:
    return iris_clf_runner.predict.run(input_series)


Writing service.py


In [ ]:
'''
BentoML 서비스 실행 - 별도의 cli나 ipynb 파일 등에서 실행

bentoml serve service.py:svc --reload
'''